# 2.2 Unsupervised Machine Learning
## This script includes the following points:
### 1. Importing Libraries and Data
### 2. Data Wrangling and Scaling
### 3. Reshaping for modeling
### 4. Creating Keras Model
### 5. Creating Confusion Matrix
### 6. Updated Example Code for LSTM (RNN Approach):

## 1. Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
#Create a path to where your data is stored.
path = r"C:\Users\Poory\OneDrive\Desktop\project\ClimateWins"

In [3]:
#Read in the European weather data and Answers Data set.

climate = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Dataset-weather-prediction-dataset-processed.csv'))
answers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
climate.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_snow_depth,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,GDANSK_cloud_cover,GDANSK_humidity,GDANSK_precipitation,GDANSK_snow_depth,GDANSK_temp_mean,GDANSK_temp_min,GDANSK_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_snow_depth,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_snow_depth,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_snow_depth,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,ROMA_cloud_cover,ROMA_wind_speed,ROMA_humidity,ROMA_pressure,ROMA_sunshine,ROMA_temp_mean,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,7.7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,5.4,0.83,1.0161,0.12,0.08,0,0.0,10.0,7.0,11.5,8,0.91,0.00,0,0.8,-0.3,1.6,7,0.91,1.0010,0.13,0.22,0,0.0,10.6,9.4,8.3,2.9,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.4,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,8.7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.0,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,0.20,0.10,0,0.0,6.9,1.1,10.4,8,4.0,0.98,0.9978,0.04,1.14,0,0.0,4.9,3.8,5.9,3,2.6,0.73,1.0152,7.1,7.8,4,4.5,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,3.8,0.76,1.0169,1.54,0.44,10.0,7.8,12.2,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84

In [6]:
answers.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
climate.shape

(22950, 170)

In [8]:
answers.shape

(22950, 16)

## 2. Data Wrangling

### Every set of observations needs to be the same length. Three of the weather stations don’t have enough and need to be dropped from the data set. You’ll find them already removed from the predictions data set

In [9]:
# Drop 3 weather stations not included in answers.
awn_col = []
for i in answers.columns.tolist():
    awn_col.append(i.split('_')[0])

awn_col = set(awn_col)
awn_col

{'BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DATE',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA'}

### These are unique columns in answers DataFrame

In [10]:
# Finding all stations in Climate DataFrame
col = climate.columns.tolist()
cli_col = [i.split('_')[0] for i in col]
cli_col = set(cli_col)

In [11]:
# Finding Stations that are not in both DataFrames
dif = []
for i in cli_col:
    if i not in awn_col:
        dif.append(i)
dif

['ROMA', 'GDANSK', 'TOURS', 'MONTH']

In [12]:
# Searching for stations that are not Awnsers DataFrame

import re

del_colum = []
for i in col:
    if re.search('GDANSK|ROMA|TOURS|MONTH', str(i)):
        del_colum.append(i)
del_colum.append('DATE')
del_colum

['MONTH',
 'GDANSK_cloud_cover',
 'GDANSK_humidity',
 'GDANSK_precipitation',
 'GDANSK_snow_depth',
 'GDANSK_temp_mean',
 'GDANSK_temp_min',
 'GDANSK_temp_max',
 'ROMA_cloud_cover',
 'ROMA_wind_speed',
 'ROMA_humidity',
 'ROMA_pressure',
 'ROMA_sunshine',
 'ROMA_temp_mean',
 'TOURS_wind_speed',
 'TOURS_humidity',
 'TOURS_pressure',
 'TOURS_global_radiation',
 'TOURS_precipitation',
 'TOURS_temp_mean',
 'TOURS_temp_min',
 'TOURS_temp_max',
 'DATE']

### These columns should be deleted from Climates DataFrame

In [13]:
answers.drop(columns=['DATE'], axis=1, inplace =True)
climate.drop(columns=del_colum, axis=1, inplace=True)
answers.shape

(22950, 15)

In [14]:
climate.shape

(22950, 147)

### Two types of observations are missing multiple years for most weather stations. Remove them

In [15]:
#Remove 2 types of observations (columns) missing multiple entries for most stations.

col = climate.columns.tolist()
obs = [i.split('_')[-1] for i in col]
obs = pd.Series(obs)
obs.value_counts()

radiation        15
precipitation    15
sunshine         15
mean             15
min              15
max              15
cover            14
humidity         14
pressure         14
speed             9
depth             6
Name: count, dtype: int64

### It seems most stations don't have information about Speed and Depth, So we can drop columns with Speed and Depth in their name

In [16]:
# Searching for columns that have 'Speed' and 'Depth' in their name in climate DataFrame
import re

del_colum2 = []
for i in col:
    if re.search('speed|depth', str(i)):
        del_colum2.append(i)
del_colum2

['BASEL_wind_speed',
 'BASEL_snow_depth',
 'DEBILT_wind_speed',
 'DUSSELDORF_wind_speed',
 'DUSSELDORF_snow_depth',
 'HEATHROW_snow_depth',
 'KASSEL_wind_speed',
 'LJUBLJANA_wind_speed',
 'MAASTRICHT_wind_speed',
 'MADRID_wind_speed',
 'MUNCHENB_snow_depth',
 'OSLO_wind_speed',
 'OSLO_snow_depth',
 'SONNBLICK_wind_speed',
 'VALENTIA_snow_depth']

### These columns should be deleted from Climates DataFrame too

In [17]:
climate.drop(columns=del_colum2, axis=1, inplace=True)
climate.shape

(22950, 132)

### There are three individual observations that need to be filled in. Assume nearby stations have similar weather, then pick one to copy the data from (Ljubljana is near Kassel, Sonnblick is near Munchen, and Olso is close enough to Stockholm).

In [18]:
# Fill in 3 individual observations assuming nearby stations have similar weather.


col = climate.columns.tolist()
complete_stations = []
for i in col:
    if re.search('cover|humidity|pressure', str(i)):
        complete_stations.append(i)
complete_stations = [i.split('_')[0] for i in complete_stations]
complete_stations = pd.Series(complete_stations)
complete_stations.value_counts()

BASEL         3
BELGRADE      3
BUDAPEST      3
DEBILT        3
DUSSELDORF    3
HEATHROW      3
LJUBLJANA     3
MAASTRICHT    3
MADRID        3
OSLO          3
SONNBLICK     3
VALENTIA      3
KASSEL        2
MUNCHENB      2
STOCKHOLM     2
Name: count, dtype: int64

In [19]:
# List all column names from the DataFrame
col = climate.columns.tolist()

# Create a list to hold columns that match specific weather measurements
complete_obs = []
for i in col:
    if re.search('cover|humidity|pressure', str(i)):
        complete_obs.append(i)

# Extract station names and measurements from the matched columns
complete_obs = [[i.split('_')[0], i.split('_')[-1]] for i in complete_obs]

# Find stations with incomplete measurements
uncomplete_obs = []
station_measurement_counts = {}

# Count occurrences of measurements for each station
for station, measurement in complete_obs:
    if station not in station_measurement_counts:
        station_measurement_counts[station] = 0
    station_measurement_counts[station] += 1

# Identify stations with fewer than 3 measurements
for station, count in station_measurement_counts.items():
    if count < 3:
        uncomplete_obs.append(station)

# Output the stations with incomplete observations
uncomplete_obs

['KASSEL', 'MUNCHENB', 'STOCKHOLM']

In [20]:
climate.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.01,3.0,6.8,4.6,9.9,7,0.95,1.0161,0.12,0.07,0.0,7.1,6.9,9.1,8,0.96,1.0166,0.15,0.07,0.

In [21]:
climate['STOCKHOLM_humidity'] = climate['OSLO_humidity']
climate['MUNCHENB_pressure'] = climate['SONNBLICK_pressure']
climate['KASSEL__cloud_cover'] = climate['LJUBLJANA_cloud_cover']

col = climate.columns.tolist()
obs = [i.split('_')[-1] for i in col]
obs = pd.Series(obs)
obs.value_counts()

cover            15
humidity         15
pressure         15
radiation        15
precipitation    15
sunshine         15
mean             15
min              15
max              15
Name: count, dtype: int64

In [22]:
climate.shape

(22950, 135)

In [23]:
answers.shape

(22950, 15)

In [24]:
# Export Cleaned Data Sets
climate.to_csv(os.path.join(path, '02 Data', 'Prepared data', 'Unsupervised', 'climate_clean.csv'), index=False)
answers.to_csv(os.path.join(path, '02 Data', 'Prepared data', 'Unsupervised', 'answers_clean.csv'), index=False)

## 3. Reshaping for modeling

In [25]:
X = np.array(climate)
y = np.array(answers)

In [26]:
X = X.reshape(-1,15,9)

In [27]:
X.shape

(22950, 15, 9)

In [28]:
y.shape

(22950, 15)

In [29]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Creating Keras Model

In [30]:
epochs = 20
batch_size = 32
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

C:\Users\Poory\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 14, 64)              │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 14, 16)              │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 7, 16)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 112)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           1,695 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Compiling and Running

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
503/503 - 4s - 8ms/step - accuracy: 0.0847 - loss: 2122.1404
Epoch 2/20
503/503 - 2s - 3ms/step - accuracy: 0.1160 - loss: 21557.0371
Epoch 3/20
503/503 - 1s - 3ms/step - accuracy: 0.1131 - loss: 71316.5000
Epoch 4/20
503/503 - 1s - 3ms/step - accuracy: 0.1163 - loss: 153314.4219
Epoch 5/20
503/503 - 2s - 3ms/step - accuracy: 0.1180 - loss: 278780.8438
Epoch 6/20
503/503 - 2s - 3ms/step - accuracy: 0.1186 - loss: 442403.6250
Epoch 7/20
503/503 - 2s - 3ms/step - accuracy: 0.1209 - loss: 641010.0000
Epoch 8/20
503/503 - 2s - 3ms/step - accuracy: 0.1244 - loss: 910291.6250
Epoch 9/20
503/503 - 2s - 3ms/step - accuracy: 0.1200 - loss: 1192178.6250
Epoch 10/20
503/503 - 2s - 3ms/step - accuracy: 0.1222 - loss: 1531874.7500
Epoch 11/20
503/503 - 2s - 3ms/step - accuracy: 0.1224 - loss: 1913702.3750
Epoch 12/20
503/503 - 2s - 3ms/step - accuracy: 0.1209 - loss: 2314008.2500
Epoch 13/20
503/503 - 2s - 3ms/step - accuracy: 0.1153 - loss: 2733313.2500
Epoch 14/20
503/503 - 2s - 3ms/st

In [33]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0551 - loss: 6835244.5000
Test Loss: 6852701.0
Test Accuracy: 0.05403050035238266


In [34]:
epochs = 40
batch_size = 32
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)


loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Epoch 1/40
503/503 - 4s - 7ms/step - accuracy: 0.1208 - loss: 3370.4910
Epoch 2/40
503/503 - 2s - 4ms/step - accuracy: 0.1311 - loss: 34103.2617
Epoch 3/40
503/503 - 2s - 3ms/step - accuracy: 0.1390 - loss: 111998.3984
Epoch 4/40
503/503 - 2s - 4ms/step - accuracy: 0.1414 - loss: 259793.8125
Epoch 5/40
503/503 - 2s - 3ms/step - accuracy: 0.1413 - loss: 484470.9375
Epoch 6/40
503/503 - 2s - 3ms/step - accuracy: 0.1381 - loss: 775374.3125
Epoch 7/40
503/503 - 2s - 3ms/step - accuracy: 0.1368 - loss: 1191266.7500
Epoch 8/40
503/503 - 2s - 4ms/step - accuracy: 0.1350 - loss: 1598695.6250
Epoch 9/40
503/503 - 2s - 4ms/step - accuracy: 0.1328 - loss: 2183739.7500
Epoch 10/40
503/503 - 2s - 3ms/step - accuracy: 0.1322 - loss: 2820246.2500
Epoch 11/40
503/503 - 2s - 4ms/step - accuracy: 0.1252 - loss: 3496700.2500
Epoch 12/40
503/503 - 2s - 4ms/step - accuracy: 0.1232 - loss: 4381720.0000
Epoch 13/40
503/503 - 2s - 3ms/step - accuracy: 0.1186 - loss: 5275260.5000
Epoch 14/40
503/503 - 2s - 3ms

In [35]:
epochs = 40
batch_size = 32
n_hidden = 256

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)


loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Epoch 1/40
503/503 - 4s - 7ms/step - accuracy: 0.1101 - loss: 8111.6934
Epoch 2/40
503/503 - 2s - 4ms/step - accuracy: 0.1066 - loss: 90675.1328
Epoch 3/40
503/503 - 2s - 4ms/step - accuracy: 0.1107 - loss: 277649.3438
Epoch 4/40
503/503 - 2s - 4ms/step - accuracy: 0.1180 - loss: 640772.8750
Epoch 5/40
503/503 - 2s - 4ms/step - accuracy: 0.1211 - loss: 1149539.3750
Epoch 6/40
503/503 - 2s - 4ms/step - accuracy: 0.1214 - loss: 1750079.2500
Epoch 7/40
503/503 - 2s - 4ms/step - accuracy: 0.1167 - loss: 2613904.7500
Epoch 8/40
503/503 - 2s - 4ms/step - accuracy: 0.1093 - loss: 3441427.0000
Epoch 9/40
503/503 - 2s - 4ms/step - accuracy: 0.1037 - loss: 4280292.5000
Epoch 10/40
503/503 - 2s - 4ms/step - accuracy: 0.1084 - loss: 5240577.0000
Epoch 11/40
503/503 - 2s - 4ms/step - accuracy: 0.1057 - loss: 5809883.0000
Epoch 12/40
503/503 - 2s - 4ms/step - accuracy: 0.1107 - loss: 6567540.5000
Epoch 13/40
503/503 - 2s - 4ms/step - accuracy: 0.1096 - loss: 7261116.0000
Epoch 14/40
503/503 - 2s - 4

In [36]:
# Using ChatGPT for optimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Hyperparameters
epochs = 30
batch_size = 32
n_hidden = 128
learning_rate = 0.0001

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

# Compile with a lower learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/30
503/503 - 5s - 10ms/step - accuracy: 0.0406 - loss: 14.6049
Epoch 2/30
503/503 - 3s - 5ms/step - accuracy: 0.0401 - loss: 32.6966
Epoch 3/30
503/503 - 3s - 5ms/step - accuracy: 0.0373 - loss: 91.7584
Epoch 4/30
503/503 - 3s - 5ms/step - accuracy: 0.0405 - loss: 174.3920
Epoch 5/30
503/503 - 3s - 5ms/step - accuracy: 0.0457 - loss: 258.2512
Epoch 6/30
503/503 - 3s - 5ms/step - accuracy: 0.0446 - loss: 340.2732
Epoch 7/30
503/503 - 3s - 5ms/step - accuracy: 0.0459 - loss: 429.2524
Epoch 8/30
503/503 - 3s - 5ms/step - accuracy: 0.0465 - loss: 529.2323
Epoch 9/30
503/503 - 3s - 5ms/step - accuracy: 0.0478 - loss: 631.5597
Epoch 10/30
503/503 - 3s - 5ms/step - accuracy: 0.0509 - loss: 736.3328
Epoch 11/30
503/503 - 3s - 5ms/step - accuracy: 0.0494 - loss: 848.5302
Epoch 12/30
503/503 - 2s - 5ms/step - accuracy: 0.0518 - loss: 968.1545
Epoch 13/30
503/503 - 3s - 5ms/step - accuracy: 0.0537 - loss: 1091.4501
Epoch 14/30
503/503 - 3s - 5ms/step - accuracy: 0.0541 - loss: 1218.0662
E

In [37]:
# Using ChatGPT for optimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Hyperparameters
epochs = 50
batch_size = 64
n_hidden = 64
learning_rate = 0.00005

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

# Compile with a lower learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Epoch 1/50
252/252 - 4s - 16ms/step - accuracy: 0.0476 - loss: 12.6455
Epoch 2/50
252/252 - 1s - 6ms/step - accuracy: 0.0950 - loss: 11.6182
Epoch 3/50
252/252 - 1s - 6ms/step - accuracy: 0.1123 - loss: 12.8781
Epoch 4/50
252/252 - 1s - 6ms/step - accuracy: 0.1176 - loss: 14.7380
Epoch 5/50
252/252 - 1s - 6ms/step - accuracy: 0.1215 - loss: 16.8221
Epoch 6/50
252/252 - 1s - 6ms/step - accuracy: 0.1216 - loss: 19.1358
Epoch 7/50
252/252 - 1s - 6ms/step - accuracy: 0.1211 - loss: 21.8648
Epoch 8/50
252/252 - 1s - 6ms/step - accuracy: 0.1193 - loss: 24.9687
Epoch 9/50
252/252 - 1s - 6ms/step - accuracy: 0.1094 - loss: 29.3028
Epoch 10/50
252/252 - 1s - 5ms/step - accuracy: 0.1079 - loss: 34.3333
Epoch 11/50
252/252 - 1s - 6ms/step - accuracy: 0.1162 - loss: 39.7134
Epoch 12/50
252/252 - 1s - 6ms/step - accuracy: 0.1239 - loss: 45.7138
Epoch 13/50
252/252 - 1s - 6ms/step - accuracy: 0.1264 - loss: 51.5774
Epoch 14/50
252/252 - 1s - 5ms/step - accuracy: 0.1323 - loss: 58.5688
Epoch 15/50
25

## 6. Creating Confusion Matrix

In [38]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [39]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         309       572       560     212         604       402       2   
BELGRADE      116       336       203      38          29        76       0   
BUDAPEST       14        23        24      21          16        24       0   
DEBILT          1        15        15       2           3        21       0   
DUSSELDORF      2         1         1       6           0         5       0   
HEATHROW        3         4         4      20           9        17       0   
KASSEL          0         3         1       0           0         2       0   
LJUBLJANA       1         6         3       6          11         7       0   
MAASTRICHT      0         0         0       0           2         2       0   
MADRID         40        29        11      22         243        29       0   
MUNCHENB   

## ChatGPT Version
### 7. Updated Example Code for LSTM (RNN Approach):

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Hyperparameters
epochs = 50
batch_size = 64
n_hidden = 128
learning_rate = 0.00005

# Define the LSTM model
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.2))
model.add(Dense(n_classes, activation='softmax'))

# Compile with a lower learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


C:\Users\Poory\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
252/252 - 9s - 36ms/step - accuracy: 0.0529 - loss: 9.1008
Epoch 2/50
252/252 - 6s - 24ms/step - accuracy: 0.1064 - loss: 9.9883
Epoch 3/50
252/252 - 6s - 24ms/step - accuracy: 0.1146 - loss: 10.4076
Epoch 4/50
252/252 - 6s - 24ms/step - accuracy: 0.1150 - loss: 10.4494
Epoch 5/50
252/252 - 6s - 24ms/step - accuracy: 0.1147 - loss: 10.3686
Epoch 6/50
252/252 - 6s - 24ms/step - accuracy: 0.1084 - loss: 10.3345
Epoch 7/50
252/252 - 6s - 24ms/step - accuracy: 0.0980 - loss: 10.3114
Epoch 8/50
252/252 - 6s - 24ms/step - accuracy: 0.0919 - loss: 10.2918
Epoch 9/50
252/252 - 6s - 24ms/step - accuracy: 0.0827 - loss: 10.2511
Epoch 10/50
252/252 - 6s - 22ms/step - accuracy: 0.0677 - loss: 10.2364
Epoch 11/50
252/252 - 6s - 24ms/step - accuracy: 0.0647 - loss: 10.1644
Epoch 12/50
252/252 - 6s - 24ms/step - accuracy: 0.0561 - loss: 10.1302
Epoch 13/50
252/252 - 6s - 24ms/step - accuracy: 0.0535 - loss: 10.0805
Epoch 14/50
252/252 - 6s - 24ms/step - accuracy: 0.0526 - loss: 10.0571
Epo

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Hyperparameters
epochs = 100
batch_size = 64
n_hidden = 128
learning_rate = 0.0001

# Define the LSTM model
model = Sequential()
model.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(n_classes, activation='softmax'))

# Compile with a higher learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/100
252/252 - 15s - 58ms/step - accuracy: 0.1007 - loss: 9.3442
Epoch 2/100
252/252 - 9s - 38ms/step - accuracy: 0.0790 - loss: 9.7856
Epoch 3/100
252/252 - 10s - 38ms/step - accuracy: 0.0619 - loss: 9.7307
Epoch 4/100
252/252 - 10s - 38ms/step - accuracy: 0.0576 - loss: 9.6539
Epoch 5/100
252/252 - 10s - 38ms/step - accuracy: 0.0632 - loss: 9.5925
Epoch 6/100
252/252 - 9s - 37ms/step - accuracy: 0.0624 - loss: 9.5310
Epoch 7/100
252/252 - 9s - 37ms/step - accuracy: 0.0617 - loss: 9.5060
Epoch 8/100
252/252 - 9s - 38ms/step - accuracy: 0.0617 - loss: 9.4750
Epoch 9/100
252/252 - 10s - 38ms/step - accuracy: 0.0622 - loss: 9.4584
Epoch 10/100
252/252 - 9s - 38ms/step - accuracy: 0.0640 - loss: 9.4400
Epoch 11/100
252/252 - 9s - 37ms/step - accuracy: 0.0640 - loss: 9.4129
Epoch 12/100
252/252 - 10s - 38ms/step - accuracy: 0.0617 - loss: 9.4107
Epoch 13/100
252/252 - 10s - 38ms/step - accuracy: 0.0613 - loss: 9.4029
Epoch 14/100
252/252 - 9s - 38ms/step - accuracy: 0.0611 - loss: 9

In [42]:
print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
Pred        BELGRADE  BUDAPEST  DEBILT  KASSEL  LJUBLJANA  MADRID  MUNCHENB  \
True                                                                          
BASEL              5         2       0       2          1    1760         1   
BELGRADE          12         3       0       6          0    1204         0   
BUDAPEST           0         2       0       0          0     237         0   
DEBILT             0         0       0       0          0      96         0   
DUSSELDORF         0         0       0       1          0      30         0   
HEATHROW           0         0       0       0          0      90         1   
KASSEL             0         0       0       0          0      13         0   
LJUBLJANA          0         0       0       0          0      70         0   
MAASTRICHT         0         0       0       1          0       5         0   
MADRID             0         0       1       0          0     406         0   
MUNCHENB  

## Scaling Data sets and trying again

In [43]:
from sklearn.preprocessing import StandardScaler

# Scaling the data
scaler = StandardScaler()
scaled_climate = scaler.fit_transform(climate)

# Convert back to DataFrame for easy readability
scaled_climate_df = pd.DataFrame(scaled_climate, columns=climate.columns)

scaled_climate_df.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,STOCKHOLM_humidity,MUNCHENB_pressure,KASSEL__cloud_cover
0,0.660514,0.826097,-0.001949,-1.101066,-0.265148,-0.902918,-0.528623,-0.845652,-0.478356,-1.206433,0.905270,0.321771,-0.743149,-0.371315,0.283983,-1.016876,-1.220210,-0.949203,-0.25524,-0.010098,-0.005876,-1.118537,-0.321868,-0.773378,-1.099163,-1.119431,-1.136839,0.713782,0.357839,-1.232471,-1.197013,0.049619,-1.077312,-0.114356,0.263326,-0.411047,1.232526,0.625746,-0.005606,-1.157575,-0.305938,-1.014908,-0.105836,0.060555,-0.410423,0.833387,1.332572,-1.376592,-1.187370,0.141032,-1.064721,-0.106469,0.393470,-1.038492,0.449867,-0.801741,-1.069690,0.747355,-0.647708,-0.182904,-0.251893,-0.466503,1.205492,1.864672,-0.008724,-1.203514,-0.406430,-1.149211,-1.370824,-1.125110,-1.578385,0.703568,0.267752,-1.043873,-1.052433,0.252496,-0.810834,-0.097084,0.382899,-0.387967,0.855172,1.887115,1.149046,-1.423735,-0.315682,-1.521762,-0.988280,-0.877723,-1.044732,-0.093824,-0.746542,-1.244144,-0.282933,-1.098059,-0.265742,-0.505735,-0.350849,1.111743,1.544023,-1.142400,-1.067870,1.925426,-1.018677,-0.186575,0.094958,-0.474865,-0.428835,-0.629430,0.095234,-1.244440,-0.649856,-0.550327,-0.124331,-0.170125,-0.063322,-0.060102,-0.003465,-1.079126,0.479131,-1.013295,-0.391072,-0.290439,-0.639538,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,0.372147,-0.668215,-0.519743,-0.752237,1.544023,0.095234,1.205492
1,0.244897,0.735760,-0.001949,-1.058108,1.658760,-0.81

In [44]:
X = np.array(scaled_climate_df)
y = np.array(answers)

In [45]:
X = X.reshape(-1,15,9)

In [46]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [47]:
# Using ChatGPT for optimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Hyperparameters
epochs = 50
batch_size = 64
n_hidden = 64
learning_rate = 0.00005

# Define the model
model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

# Compile with a lower learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

C:\Users\Poory\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
252/252 - 4s - 17ms/step - accuracy: 0.0327 - loss: 12.6249
Epoch 2/50
252/252 - 1s - 6ms/step - accuracy: 0.0945 - loss: 12.6660
Epoch 3/50
252/252 - 1s - 6ms/step - accuracy: 0.1342 - loss: 13.9549
Epoch 4/50
252/252 - 1s - 6ms/step - accuracy: 0.1485 - loss: 15.9054
Epoch 5/50
252/252 - 1s - 6ms/step - accuracy: 0.1536 - loss: 18.3522
Epoch 6/50
252/252 - 1s - 6ms/step - accuracy: 0.1410 - loss: 21.5757
Epoch 7/50
252/252 - 1s - 6ms/step - accuracy: 0.1258 - loss: 25.5013
Epoch 8/50
252/252 - 1s - 6ms/step - accuracy: 0.0955 - loss: 29.9927
Epoch 9/50
252/252 - 1s - 6ms/step - accuracy: 0.0739 - loss: 35.2628
Epoch 10/50
252/252 - 1s - 6ms/step - accuracy: 0.0653 - loss: 41.2199
Epoch 11/50
252/252 - 1s - 6ms/step - accuracy: 0.0627 - loss: 48.2318
Epoch 12/50
252/252 - 1s - 6ms/step - accuracy: 0.0603 - loss: 56.6643
Epoch 13/50
252/252 - 1s - 6ms/step - accuracy: 0.0614 - loss: 66.5953
Epoch 14/50
252/252 - 1s - 6ms/step - accuracy: 0.0606 - loss: 77.8446
Epoch 15/50
25

In [48]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         297       636       357     291         305       502      88   
BELGRADE      141       150       173      36         103       105      18   
BUDAPEST       52        17        22      13          10        17      16   
DEBILT         10         5        23       7           3         3       6   
DUSSELDORF      5         2         4       3           0         1       4   
HEATHROW       11        10         8       8           0        15       6   
KASSEL          1         1         2       0           0         0       0   
LJUBLJANA       9         8         3       3           3        10       3   
MAASTRICHT      0         3         0       0           0         2       1   
MADRID         32        53        16      41          29       140      31   
MUNCHENB   

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Hyperparameters
epochs = 100
batch_size = 64
n_hidden = 64
learning_rate = 0.0001

# Define the LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim))))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(n_classes, activation='softmax'))

# Compile with a lower learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Learning rate scheduler to reduce the learning rate if loss does not improve
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=[reduce_lr], validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/100


C:\Users\Poory\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


252/252 - 18s - 71ms/step - accuracy: 0.1247 - loss: 8.9974 - val_accuracy: 0.1763 - val_loss: 8.9059 - learning_rate: 1.0000e-04
Epoch 2/100
252/252 - 6s - 23ms/step - accuracy: 0.2040 - loss: 9.0771 - val_accuracy: 0.1943 - val_loss: 8.9247 - learning_rate: 1.0000e-04
Epoch 3/100
252/252 - 7s - 26ms/step - accuracy: 0.1975 - loss: 9.0569 - val_accuracy: 0.1834 - val_loss: 8.9106 - learning_rate: 1.0000e-04
Epoch 4/100
252/252 - 7s - 26ms/step - accuracy: 0.1949 - loss: 9.0185 - val_accuracy: 0.1836 - val_loss: 8.8678 - learning_rate: 1.0000e-04
Epoch 5/100
252/252 - 7s - 26ms/step - accuracy: 0.1922 - loss: 8.9874 - val_accuracy: 0.1846 - val_loss: 8.8349 - learning_rate: 1.0000e-04
Epoch 6/100
252/252 - 6s - 25ms/step - accuracy: 0.1930 - loss: 8.9610 - val_accuracy: 0.1885 - val_loss: 8.8024 - learning_rate: 1.0000e-04
Epoch 7/100
252/252 - 6s - 25ms/step - accuracy: 0.1955 - loss: 8.9327 - val_accuracy: 0.1890 - val_loss: 8.7896 - learning_rate: 1.0000e-04
Epoch 8/100
252/252 - 6s

In [50]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
Pred        BELGRADE  DEBILT  HEATHROW  MAASTRICHT  MADRID  MUNCHENB  \
True                                                                   
BASEL           1754       5         0           0       6         2   
BELGRADE        1169       4         0           3       8         1   
BUDAPEST         230       1         0           0       1         0   
DEBILT            96       0         0           0       0         0   
DUSSELDORF        31       0         0           0       0         0   
HEATHROW          82       0         0           2       3         0   
KASSEL            12       0         0           0       1         0   
LJUBLJANA         56       0         0           0       5         0   
MAASTRICHT         4       0         0           1       0         0   
MADRID           222       8         1           1      35         0   
MUNCHENB           3       0         0           0       2         0   
OSLO               5  